# 连接特征存储

特征存储是传统机器学习中的一个概念, 它确保进入模型的数据是最新和相关的。更多信息请参见[这里](https://www.tecton.ai/blog/what-is-a-feature-store/)。

考虑将LLM应用程序投入生产时，这个概念是非常相关的。为了个性化LLM应用程序，您可能希望将LLM与特定用户的最新信息相结合。特征存储可以是保持数据新鲜的绝佳途径，LangChain提供了一种将数据与LLMs相结合的简单方法。

在本笔记本中，我们将展示如何将提示模板连接到特征存储。基本思想是从提示模板内部调用特征存储以检索值，然后将这些值格式化为提示。


## Feast

首先，我们将使用流行的开源特征存储框架[Feast](https://github.com/feast-dev/feast)。

这假定您已经按照README中的步骤开始运行。我们将在启动中构建一个带有LLMChain的示例，以便向特定驱动程序写入关于他们最新统计数据的注释。

### 加载Feast存储

同样，这应该根据Feast README中的说明设置。

In [1]:
from feast import FeatureStore

# You may need to update the path depending on where you stored it
feast_repo_path = "../../../../../my_feature_repo/feature_repo/"
store = FeatureStore(repo_path=feast_repo_path)

### 提示信息

在这里，我们将设置一个自定义的FeastPromptTemplate。该提示模板将接受驾驶员ID，查找他们的统计信息，并将这些统计信息格式化成提示信息。

请注意，此提示模板的输入仅为`驱动程序ID`，因为那是仅用户定义的一部分（所有其他变量都在提示模板内部查找）

In [3]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [19]:
template = """Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: {conv_rate}
Acceptance rate: {acc_rate}
Average Daily Trips: {avg_daily_trips}

Your response:"""
prompt = PromptTemplate.from_template(template)

In [20]:
class FeastPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop("driver_id")
        feature_vector = store.get_online_features(
            features=[
                'driver_hourly_stats:conv_rate',
                'driver_hourly_stats:acc_rate',
                'driver_hourly_stats:avg_daily_trips'
            ],
            entity_rows=[{"driver_id": driver_id}]
        ).to_dict()
        kwargs["conv_rate"] = feature_vector["conv_rate"][0]
        kwargs["acc_rate"] = feature_vector["acc_rate"][0]
        kwargs["avg_daily_trips"] = feature_vector["avg_daily_trips"][0]
        return prompt.format(**kwargs)

In [21]:
prompt_template = FeastPromptTemplate(input_variables=["driver_id"])

In [22]:
print(prompt_template.format(driver_id=1001))

Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: 0.4745151400566101
Acceptance rate: 0.055561766028404236
Average Daily Trips: 936

Your response:


### 在链中使用

现在，我们可以在链中使用它，成功创建了一个使用特征存储支持的个性化链。

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [24]:
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt_template)

In [25]:
chain.run(1001)

"Hi there! I wanted to update you on your current stats. Your acceptance rate is 0.055561766028404236 and your average daily trips are 936. While your conversation rate is currently 0.4745151400566101, I have no doubt that with a little extra effort, you'll be able to exceed that .5 mark! Keep up the great work! And remember, even chickens can't always cross the road, but they still give it their best shot."

## Tecton

上述示例展示了如何使用Feast，这是一个流行的开源且自管理的特征存储，与LangChain集成。我们下面的示例将展示使用Tecton进行类似的集成。Tecton是一个完全管理的功能平台，可编排完整的ML功能生命周期，从转换到在线服务，具有企业级的SLA。

### 先决条件

* Tecton部署（在[https://tecton.ai](https://tecton.ai)注册）
* `TECTON_API_KEY`环境变量设置为有效的服务帐户密钥

### 定义并加载功能

我们将使用[Tecton教程](https://docs.tecton.ai/docs/tutorials/tecton-fundamentals)中的`user_transaction_counts`特性视图作为特征服务的一部分。为简单起见，我们仅使用单个特征视图；但是，更复杂的应用程序可能需要更多的特征视图来检索其提示所需的功能。

```python
user_transaction_metrics = FeatureService(
    name = "user_transaction_metrics",
    features = [user_transaction_counts]
)
```

上述特征服务预计[应用于现场工作区](https://docs.tecton.ai/docs/applying-feature-repository-changes-to-a-workspace)。对于此示例，我们将使用“prod”工作区。

In [60]:
import tecton

workspace = tecton.get_workspace("prod")
feature_service = workspace.get_feature_service("user_transaction_metrics")

### 提示信息

在这里，我们将设置一个自定义的TectonPromptTemplate。该提示模板将接受用户ID，查找他们的统计信息，并将这些统计信息格式化为提示信息。

请注意，此提示模板的输入仅为`用户ID`，因为那是仅用户定义的一部分（所有其他变量都在提示模板内部查找）

In [61]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [77]:
template = """Given the vendor's up to date transaction stats, write them a note based on the following rules:

1. If they had a transaction in the last day, write a short congratulations message on their recent sales
2. If no transaction in the last day, but they had a transaction in the last 30 days, playfully encourage them to sell more.
3. Always add a silly joke about chickens at the end

Here are the vendor's stats:
Number of Transactions Last Day: {transaction_count_1d}
Number of Transactions Last 30 Days: {transaction_count_30d}

Your response:"""
prompt = PromptTemplate.from_template(template)

In [78]:
class TectonPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        user_id = kwargs.pop("user_id")
        feature_vector = feature_service.get_online_features(join_keys={"user_id": user_id}).to_dict()
        kwargs["transaction_count_1d"] = feature_vector["user_transaction_counts.transaction_count_1d_1d"]
        kwargs["transaction_count_30d"] = feature_vector["user_transaction_counts.transaction_count_30d_1d"]
        return prompt.format(**kwargs)

In [79]:
prompt_template = TectonPromptTemplate(input_variables=["user_id"])

In [80]:
print(prompt_template.format(user_id="user_469998441571"))

Given the vendor's up to date transaction stats, write them a note based on the following rules:

1. If they had a transaction in the last day, write a short congratulations message on their recent sales
2. If no transaction in the last day, but they had a transaction in the last 30 days, playfully encourage them to sell more.
3. Always add a silly joke about chickens at the end

Here are the vendor's stats:
Number of Transactions Last Day: 657
Number of Transactions Last 30 Days: 20326

Your response:


### 在链中使用

现在，我们可以在链中使用它，成功创建了一个使用Tecton功能平台支持的个性化链。

In [81]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [82]:
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt_template)

In [83]:
chain.run("user_469998441571")

'Wow, congratulations on your recent sales! Your business is really soaring like a chicken on a hot air balloon! Keep up the great work!'

## Featureform

最后，我们将使用[Featureform](https://github.com/featureform/featureform)，这是一个开源且企业级别的功能存储，运行相同的示例。Featureform允许您与像Spark或本地这样的基础架构一起工作，定义特征转换。

### 初始化Featureform

您可以按照自述文件中的说明初始化Featureform中的转换和特征。

In [ ]:
import featureform as ff

client = ff.Client(host="demo.featureform.com")

### 提示信息

在这里，我们将设置一个自定义的FeatureformPromptTemplate。此提示模板将接受用户每次交易平均支付金额。

请注意，此提示模板的输入仅为`avg_transaction`，因为那是仅用户定义的一部分（所有其他变量都在提示模板内部查找）。

In [ ]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [ ]:
template = """Given the amount a user spends on average per transaction, let them know if they are a high roller. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the user's stats:
Average Amount per Transaction: ${avg_transcation}

Your response:"""
prompt = PromptTemplate.from_template(template)

In [ ]:
class FeatureformPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        user_id = kwargs.pop("user_id")
        fpf = client.features([("avg_transactions", "quickstart")], {"user": user_id})
        return prompt.format(**kwargs)

In [ ]:
prompt_template = FeatureformPrompTemplate(input_variables=["user_id"])

In [ ]:
print(prompt_template.format(user_id="C1410926"))

### 在链中使用

现在，我们可以在链中使用它，成功创建一个受Featureform特征平台支持的个性化链。

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [ ]:
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt_template)

In [ ]:
chain.run("C1410926")